In [1]:
! pip install -q -U bitsandbytes
! pip install -q -U datasets
! pip install -q -U transformers==4.36.0
! pip install -q -U peft
! pip install -q -U accelerate
! pip install -q -U loralib
! pip install -q -U einops
! pip install -q -U googletrans==3.1.0a0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
import json
import os
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers

from googletrans import Translator
from pprint import pprint
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig ,
    AutoModelForCausalLM ,
    AutoTokenizer ,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
MODEL_NAME = "vilm/vinallama-7b-chat"

bnb_config = BitsAndBytesConfig(
      load_in_4bit =True ,
      bnb_4bit_use_double_quant =True ,
      bnb_4bit_quant_type ="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME ,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig (
      r=16 ,
      lora_alpha=32 ,
      target_modules=[
          "q_proj",
          "up_proj",
          "o_proj",
          "k_proj",
          "down_proj",
          "gate_proj",
          "v_proj"
      ] ,
      lora_dropout=0.05 ,
      bias="none",
      task_type="CAUSAL_LM"
)

model = get_peft_model(model , config )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/709 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
generation_config=model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
prompt = """
<| im_start | > system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<| im_end | >
<| im_start | > user
{ your task }
<| im_end | >
<| im_start | > assistant
""".strip()

In [6]:
prompt = """
<| im_start | > system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<| im_end | >
<| im_start | > user
Viết một hàm tính tổng hai số trong python
<| im_end | >
<| im_start | > assistant
""".strip ()

# %%time
device = torch.device ("cuda") if torch.cuda.is_available () else torch.device("cpu")

encoding = tokenizer (prompt , return_tensors ="pt").to('cuda')
with torch.inference_mode() :
  outputs = model.generate(
    input_ids = encoding.input_ids ,
    attention_mask = encoding.attention_mask ,
    generation_config = generation_config
  )

print(tokenizer.decode(outputs[0] , skip_special_tokens=True) )

<| im_start | > system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<| im_end | >
<| im_start | > user
Viết một hàm tính tổng hai số trong python
<| im_end | >
<| im_start | > assistant
Chắc chắn! Đây là một hàm Python nhận hai số làm đầu vào và trả về tổng của chúng:
```python
def add_two_numbers(a, b):
    return a + b
```
Bạn có thể sử dụng hàm này bằng cách gọi nó với hai số làm đối số, như `add_two_numbers(3, 5)` sẽ trả về `8`.


In [ ]:
data = load_dataset('alespalla/chatbot_instruction_prompts')

Generating train split:   0%|          | 0/258042 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/64511 [00:00<?, ? examples/s]

In [ ]:
def generate_prompt ( data_point ) :
  translator = Translator()
  vn_prompt = translator.translate( data_point ['prompt'] , src='en', dest ='vi').text
  vn_response = translator.translate ( data_point['response'] , src='en', dest='vi').text

  return f"""
<| im_start | > system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<| im_end | >
<| im_start | > user
{ vn_prompt }
<| im_end | >
<| im_start | > assistant
{ vn_response }
""".strip()


In [ ]:
def generate_and_tokenize_prompt ( data_point ):
  full_prompt = generate_prompt ( data_point )
  tokenized_full_prompt = tokenizer ( full_prompt , padding =True , truncation =True )

  return tokenized_full_prompt

In [ ]:
data = data['train'].shard(num_shards =50 , index =0).filter(lambda sample : sample['response']!='' and sample['prompt']!='').shuffle().map(generate_and_tokenize_prompt)

Filter:   0%|          | 0/5161 [00:00<?, ? examples/s]

Map:   0%|          | 0/5161 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
training_args = transformers.TrainingArguments(
  per_device_train_batch_size =4 ,
  gradient_accumulation_steps =4 ,
  num_train_epochs =1 ,
  learning_rate =2e-4 ,
  fp16 =True ,
  save_total_limit =3 ,
  logging_steps =1 ,
  output_dir ="experiments",
  optim ="paged_adamw_8bit",
  lr_scheduler_type ="cosine",
  warmup_ratio =0.05 ,
)

trainer = transformers.Trainer(
  model =model ,
  train_dataset =data ,
  args = training_args ,
  data_collator = transformers.DataCollatorForLanguageModeling( tokenizer , mlm =False )
)
model.config.use_cache = False
trainer.train()

Step,Training Loss
1,1.262200
2,1.291600
3,1.219000
4,1.222800
5,1.036600
6,1.132900
7,1.151500
8,1.137700
9,1.163700
10,1.134600


TrainOutput(global_step=322, training_loss=1.0363463031949465, metrics={'train_runtime': 5622.677, 'train_samples_per_second': 0.918, 'train_steps_per_second': 0.057, 'total_flos': 5.524862942950195e+16, 'train_loss': 1.0363463031949465, 'epoch': 1.0})

In [ ]:
# %% time
device = torch.device ("cuda") if torch.cuda.is_available () else torch.device("cpu")

prompt = """
<| im_start | > system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<| im_end | >
<| im_start | > user
Mô tả về thành phố Đà Lạt
<| im_end | >
<| im_start | > assistant
""".strip()

encoding = tokenizer(prompt , return_tensors ="pt").to('cuda')
with torch.inference_mode () :
  outputs = model.generate(
    input_ids = encoding.input_ids ,
    attention_mask = encoding.attention_mask ,
    generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens = True))

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<| im_start | > system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<| im_end | >
<| im_start | > user
Mô tả về thành phố Đà Lạt
<| im_end | >
<| im_start | > assistant
Đà Lạt là một thành phố đẹp như tranh vẽ nằm ở vùng cao nguyên phía Nam nước Pháp, nổi tiếng với những vườn hoa tuyệt đẹp, những con phố cổ kính và những ngôi nhà châu Âu. Thành phố này là nơi có nhiều hoạt động giải trí, bao gồm đi bộ đường dài, đạp xe, câu cá, ngắm cảnh và thư giãn. Đà Lạt cũng là nơi có nhiều nhà hàng và quán cà phê, nơi du khách có thể thưởng thức những món ăn ngon của địa phương. Thành phố cũng có nhiều điểm tham quan, chẳng hạn như Vườn hoa, Bảo tàng Điêu khắc, Bảo tàng nghệ thuật và Bảo tàng địa chất. Đà Lạt là một điểm đến hoàn hảo cho những người muốn thư giãn và khám phá một nơi tuyệt đẹp.
